In [22]:
from dataclasses import dataclass
import pandas as pd
import numpy as np
import os
import math
import scipy
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats

data=pd.read_csv("C:/Users/KatherineRidley/Mint Diagnostics Ltd/Mint Drive - Documents/Data/IVI_serumvsaliva_KR/IVI results July (version 1).csv")
print(data.columns)
data.loc[(data['Progesterone blood (ng/mL)'].str.contains('>')) | 
(data['Progesterone blood (ng/mL)'].str.contains('<'))| 
(data['Progesterone blood (ng/mL)'].str.contains('NR')), 'Progesterone blood (ng/mL)'] = np.nan
data.loc[(data['Estradiol blood (pcg/mL)'].str.contains('>')) | (data['Estradiol blood (pcg/mL)'].str.contains('<')) | 
(data['Estradiol blood (pcg/mL)'].str.contains('NR')), 'Estradiol blood (pcg/mL)'] = np.nan
data.loc[(data['Progesterone (pg/mL)'].str.contains('>')) | 
(data['Progesterone (pg/mL)'].str.contains('<'))| 
(data['Progesterone (pg/mL)'].str.contains('NR')), 'Progesterone (pg/mL)'] = np.nan
data.loc[(data['Estradiol (pg/mL)'].str.contains('>')) | (data['Estradiol (pg/mL)'].str.contains('<')) | 
(data['Estradiol (pg/mL)'].str.contains('NR')), 'Estradiol (pg/mL)'] = np.nan
#print(data.head())

#columns: Samples, Estradiol, Progesterone, Comments, Shipment, Patient ID, Group, Date, Type, Sample, Time

#new_dict={'Samples':[],'Estradiol (pg/mL)':[],'Progesterone (pg/mL)':[],'Comments':[],'Shipment':[],'Patient ID':[],'Group':[],'Date':[],'Type':[],'Sample':[],'Time':[]}

data_s=data.copy()
data_b=data.copy()

data_s['Type']='Saliva'
data_b['Type']='Blood'
data_b.drop(['Progesterone (pg/mL)', 'Estradiol (pg/mL)'], axis=1, inplace=True)
data_s.drop(['Progesterone blood (ng/mL)', 'Estradiol blood (pcg/mL)'], axis=1, inplace=True)

data_b.rename({'Progesterone blood (ng/mL)':'Progesterone (pg/mL)', 'Estradiol blood (pcg/mL)':'Estradiol (pg/mL)'}, axis='columns', inplace=True)
data_b=data_b.drop(data_b[(data_b['Type']=='Blood') & (data_b['Progesterone (pg/mL)']=='-')].index)
#data_b['Progesterone (pg/mL)']=data_b['Progesterone (pg/mL)'].astype(float)*1000
data_b.loc[data_b['Progesterone (pg/mL)']=='NR', 'Progesterone (pg/mL)'] = np.nan
data_s.loc[data_s['Progesterone (pg/mL)']=='NR', 'Progesterone (pg/mL)'] = np.nan
data_s.reset_index()
data_b.reset_index()
print(data_b.columns)


data= pd.concat([data_s.reset_index(),data_b.reset_index()], axis=0)
data=data.loc[:,~data.columns.str.contains("Unnamed")]
data.rename({'Sample list (excluding extra samples)':'Samples'}, axis='columns', inplace=True)
data['Shipment']=3
print(data.reset_index())






Index(['Sample list (excluding extra samples)', 'Comments',
       'Estradiol (pg/mL)', 'Progesterone (pg/mL)', 'Unnamed: 4',
       'Estradiol blood (pcg/mL)', 'Progesterone blood (ng/mL)', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')
Index(['Sample list (excluding extra samples)', 'Comments', 'Unnamed: 4',
       'Estradiol (pg/mL)', 'Progesterone (pg/mL)', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Type'],
      dtype='object')
     level_0  index                   Samples Comments Estradiol (pg/mL)  \
0          0      0   S1F A3 02/05/2022 08:30      NaN              3.04   
1          1      1   S1C A3 02/05/2022 19:52      NaN              3.68   
2          2      2   S2F A3 09/05/2022 08:45      NaN              4.26   
3          3      3   S2C A3 09/05/2022 12:20      NaN              5.43   
4          4      4   S3F A3 14/05/2022 07:15      NaN

In [23]:
#regex barcode
                       
data['Sample'] = data['Samples'].str.split(' ', expand=True)[0]
#data.loc['Patient ID'] = data['Samples'].str.split(' ', expand=True)[1]
data['Group'] = data['Samples'].str.extract('\s(\D{1})(\d*)', expand=True)[0]
data['Patient ID'] = data['Samples'].str.extract('\s(\D{1})(\d*)', expand=True)[1] 
data['Date'] = data['Samples'].str.split(' ', expand=True)[2]
data['Time'] = data['Samples'].str.split(' ', expand=True)[3]

print(data)

     index                   Samples Comments Estradiol (pg/mL)  \
0        0   S1F A3 02/05/2022 08:30      NaN              3.04   
1        1   S1C A3 02/05/2022 19:52      NaN              3.68   
2        2   S2F A3 09/05/2022 08:45      NaN              4.26   
3        3   S2C A3 09/05/2022 12:20      NaN              5.43   
4        4   S3F A3 14/05/2022 07:15      NaN             11.26   
..     ...                       ...      ...               ...   
104    211  S2C A88 07/05/2022 08:35      NaN               977   
105    213  S3C A88 11/05/2022 18:55      NaN              2990   
106    215  S1C A90 17/05/2022 13:00      NaN               743   
107    217  S2C A90 20/05/2022 11:45      NaN              2092   
108    219  S3C A90 23/05/2022 12:25      NaN              2515   

    Progesterone (pg/mL)    Type  Shipment Sample Group Patient ID  \
0                 25.985  Saliva         3    S1F     A          3   
1                 93.927  Saliva         3    S1C     A

In [24]:
#concat 1 and 2 and 3
#data12 results collated 
data12=pd.read_csv("C:/Users/KatherineRidley/Mint Diagnostics Ltd/Mint Drive - Documents/Data/IVI_serumvsaliva_KR/Results_1_2_collated_KR.csv")
data_ALL= pd.concat([data12.reset_index(),data.reset_index()], axis=0)
#data_ALL['Patient ID']=data_ALL['Patient ID'].astype(int)

#print(data_ALL[data_ALL['Patient ID'].isna()])

data_ALL.to_csv("C:/Users/KatherineRidley/Mint Diagnostics Ltd/Mint Drive - Documents/Data/IVI_serumvsaliva_KR/Results_1_2_3_collated_KR.csv", index=False)
#data_ALL


1
